# train wmn using additional generated images

In [1]:
# add parent dir to syspath
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
#root_dir = os.path.dirname(parent_dir)
sys.path.insert(0, parent_dir)

In [2]:
from evaluationModels.scut_wmn.FCSRN import fcsrn
from evaluationModels.scut_wmn import wmn_helpers
from evaluationModels import evalFunctions

from imageGenerators import load_digits, load_realdata
from imageGenerators.imgGen_simple import synth_generator
from models.cyclegan_modified.cyclegan import cyclegan

import tensorflow as tf
import cv2
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import time
import re
import random
import pickle

import pprint
pp = pprint.PrettyPrinter(indent=4)

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
# assert model is running on gpu
from tensorflow.python.client import device_lib
devlist = str(device_lib.list_local_devices())
assert "GPU:0" in devlist

In [4]:
# Path to i2i-model to use for imagegeneration
CHECKPOINTPATH = Path("D:/m2/savedmodels/checkpoints_cyclegan_modified_midstate")
EPOCH_OF_MODEL = 10

generate_normalDigits = False
generate_midstateDigits = False
targetCount = None 
model_savepath = None


model_savepath = Path("D:/m2/savedmodels/FCSRN_with_initializer")
targetCount = 0

In [5]:
#generate_normalDigits = True; generate_midstateDigits = False
#model_savepath = Path("D:/m2/evaluations/fcsrn/add_normal")
#targetCount = 1000

In [6]:
generate_normalDigits = False; generate_midstateDigits = True
model_savepath = Path("D:/m2/evaluations/fcsrn/add_midstate2")
targetCount = 1000

In [7]:
assert model_savepath != None
print(model_savepath)

D:\m2\evaluations\fcsrn\add_midstate2


In [8]:
# dimensions of input to wmn-model
HEIGHT = 48; WIDTH = 160; CHANNELS = 3
IMAGE_SHAPE = (HEIGHT, WIDTH, CHANNELS)

EPOCHS = 100
BATCH_SIZE = 100

N_DIGITS = 5   # digits per label
N_CLASSES = 20 # there are 10 digits and 10 midstate-digits

# get shape of inputs for i2i-model
inputshapePath = CHECKPOINTPATH / "inputshape"
cyclegan_inputShape = [int(s) for s in inputshapePath.read_text().split(",")]


# load original data + labels

In [9]:
images_train, labels_train, images_test, labels_test =  wmn_helpers.load_wmn_traindata(WIDTH, HEIGHT, keepRatio=True)
images_easy, labels_easy = wmn_helpers.load_easySamples(WIDTH, HEIGHT, keepRatio=True)

print(images_train.shape)
print(labels_train.shape)

C:\Users\andre\jupyter_ws\evaluationModels\scut_wmn\wmn_helpers.py:52: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv(trainfile, sep="[ ,]" ,header=None)
C:\Users\andre\jupyter_ws\evaluationModels\scut_wmn\wmn_helpers.py:62: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv(testfile, sep="[ ,]" ,header=None)
C:\Users\andre\jupyter_ws\evaluationModels\scut_wmn\wmn_helpers.py:78: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can 

(4000, 48, 160, 3)
(4000, 5)


# Generate Images

In [10]:
####
# find out how many of each digit to generate by counting each digit in trainingdata.
# Returns: Map from digit to number to generate.
####
def countDigits(labels, targetDigitCount, useNormalDigits=True, useMidstateDigits=False, printResult=False):
    print("counting digits")
    if not useNormalDigits and not useMidstateDigits:
        return {}
    firstDigit = 0 if useNormalDigits else 10
    lastdigit = 20 if useMidstateDigits else 10
    # count how often each digit occurs in traindata
    digits, counts = np.unique(labels_train.flatten(), return_counts=True)
    
    # calculate how many of each digit to generate so that each occurs at least <targetDigitCount> times.
    n_digitsToGenerate = {}
    for digit in range(firstDigit, lastdigit):
        n_digitsToGenerate[digit] = targetDigitCount - counts[digit] if counts[digit] < targetDigitCount else 0
        
    # the total number of generated digits must fit into labels of length <N_DIGITS>
    sum_additionalDigits = sum(n_digitsToGenerate.values())
    n_digitsToGenerate[np.random.randint(firstDigit, lastdigit)] += (N_DIGITS - sum_additionalDigits % N_DIGITS)
    if printResult:
        print("targetcount: %d"%(targetDigitCount))
        print("additional digits:")
        for key, value in sorted(n_digitsToGenerate.items(), key=lambda item: item[0]):
            print("%d: %d"%(key,value))
        sum_additionalDigits = sum(n_digitsToGenerate.values())
        print("total additional digits: %d" % (sum_additionalDigits) )
        print("total additional images: %f"% (sum_additionalDigits/N_DIGITS) )
    return n_digitsToGenerate

In [11]:
####
# generate synthetic images + labels from map n_toGenerate
####
def generate_synthetic(n_toGenerate):
    print("generating synthetic images")
    # prepare labels
    sum_additionalDigits = sum(n_toGenerate.values())
    n_additionalImages = int(sum_additionalDigits / N_DIGITS)
    labels_gen = np.concatenate([
        np.full(shape=count, fill_value=digit) for digit, count in n_toGenerate.items()
    ])
    np.random.shuffle(labels_gen)
    labels_gen = np.reshape(labels_gen, (n_additionalImages, N_DIGITS) )
    
    # generate images from labels
    ####
    # parameters for generator
    ####
    imageHeight = cyclegan_inputShape[0]
    imageWidth = cyclegan_inputShape[1]
    
    # margins between digits, padding around resultimage 
    margins = [30 for _ in range(0, N_DIGITS - 1)]
    padding = (0,0, 1,1) # top,bottom, left, right
    # margin between digits in same column
    vertical_margin = 20
    ####
    # ranges for how far digits can scroll up or down, 0 meaning no scrolling, 0.5 meaning halfway to next digit
    range_normal=(-0.1,0.1)
    range_midstate=(0.3,0.7)

    ####
    # whether to draw vertical seperator-lines between digits
    draw_seperators = False

    ####
    # generate images
    fontnumber = 29 # number of the font to use
    dsPath = Path("C:/Users/andre/Desktop/m/datasets/Chars74K/English/Fnt")
    digitImages = load_digits.load_char74k(dsPath, fonts=[fontnumber - 1])  # index starts at zero, fontnumber at 1
    synthGen = synth_generator(digitImages, vertical_margin)
    images_synthetic = synthGen.generate_images(labels_gen, margins, padding, imageWidth, imageHeight, draw_vertical_seperators=draw_seperators, range_normal=range_normal,range_midstate=range_midstate)
    
    print("synthetic images:")
    for i in range(20):
        image = images_synthetic[i]
        plt.imshow(image, cmap="gray")
        plt.show()
    
    return (images_synthetic, labels_gen)

In [12]:
# translate synthetic images into realistic images using cyclegan
def translate_images(synthImages, width=WIDTH, height=HEIGHT, keepRatio = True):
    print("translating synthetic images to realistic images")
    ####
    # load i2i model
    cgModel = cyclegan(cyclegan_inputShape, checkpoint_path=CHECKPOINTPATH, load_checkpoint_after_epoch=EPOCH_OF_MODEL)
    # prepare input for cgan
    cgan_input = tf.data.Dataset.from_tensor_slices(synthImages)\
                .map(cgModel.preprocess_input, num_parallel_calls=AUTOTUNE)\
                .cache()\
                .batch(1)
    ####
    # predict realistic images
    translated_images = cgModel.gen_AtoB.predict(cgan_input)
    
    # denormalize
    translated_images = (translated_images + 1) * 127.5
    
    # resize to inputshape of wmn-model
    if keepRatio:
        translated_images = np.array([
            wmn_helpers.resize_withPadding(image, width, height) for image in translated_images
        ])
    else:       
        translated_images = np.array([
            cv2.resize(image, (width, height)) for image in translated_images
        ])
        
    print("translated images:")
    for i in range(20):
        image = translated_images[i]
        plt.imshow(image.astype(int), cmap="gray")
        plt.show()
    
    return translated_images

In [13]:
def appendAndShuffle(imagesA, labelsA, imagesB, labelsB):
    print("adding translated images to original traindata")
    # append
    imagesA = np.append(imagesA, imagesB, axis=0)
    labelsA = np.append(labelsA, labelsB, axis=0)
    # zip and shuffle
    trainset = list(zip(imagesA, labelsA))
    random.shuffle(trainset)
    # unzip
    imagesA, labelsA = zip(*trainset)
    return (np.array(imagesA), np.array(labelsA))

In [14]:
def generate_and_add_to_traindata(images_train=images_train, labels_train=labels_train, targetDigitCount=1000, useNormalDigits=False, useMidstateDigits=True):
    # calc how many digits to generate
    n_toGenerate = countDigits(labels_train, targetDigitCount, useNormalDigits=useNormalDigits, useMidstateDigits=useMidstateDigits, printResult=True)
    # generate synthethic images
    images_synthetic, labels_synthetic = generate_synthetic(n_toGenerate)
    # translate synthethic images to realistic images
    images_translated = translate_images(images_synthetic)
    
    # append translated dataset to original dataset, shuffle
    images_train, labels_train = appendAndShuffle(images_train, labels_train, images_translated, labels_synthetic)   
    
    for i in range(30):
        print(labels_synthetic[i])
        plt.imshow(images_synthetic[i].astype(int), cmap="gray")
        plt.show()
        plt.imshow(images_translated[i].astype(int), cmap="gray")
        plt.show()
    
    return images_train, labels_train, n_toGenerate

# Make tf-Datasets

In [15]:
def make_datasets(images, labels):
    X = tf.data.Dataset.from_tensor_slices(images)\
        .cache() \
        .batch(BATCH_SIZE, drop_remainder=True)
    y = tf.data.Dataset.from_tensor_slices(labels)\
        .cache() \
        .batch(BATCH_SIZE, drop_remainder=True)
    return (X, y)

# build/load model

In [16]:
modelExists = (model_savepath / "checkpoint").exists()
model = fcsrn(IMAGE_SHAPE, checkpoint_path=model_savepath)

loaded checkpoint:  D:\m2\evaluations\fcsrn\add_midstate2\epoch-100


# train 

In [17]:
len(images_train)

4000

In [18]:
# avoid accidentally continuing training when loading trained model
if not modelExists:    
    images_train, labels_train, n_toGenerate = generate_and_add_to_traindata(targetDigitCount=targetCount, useNormalDigits=generate_normalDigits, useMidstateDigits=generate_midstateDigits)
    train_X, train_y = make_datasets(images_train, labels_train)
    
    n_batches = len(images_train) / BATCH_SIZE
    print("Training %d batches of size %d for %d epochs" % (n_batches, BATCH_SIZE, EPOCHS) )
    model.train(train_X, train_y, EPOCHS, BATCH_SIZE)
else:
    n_toGenerate = countDigits(labels_train, targetCount, useNormalDigits=generate_normalDigits, useMidstateDigits=generate_midstateDigits, printResult=True)
    print("not training because loaded trained model.")

counting digits
targetcount: 1000
additional digits:
10: 843
11: 790
12: 785
13: 660
14: 835
15: 816
16: 798
17: 782
18: 755
19: 781
total additional digits: 7845
total additional images: 1569.000000
not training because loaded trained model.


# Evaluate 

In [19]:
test_X, test_y = make_datasets(images_test, labels_test)
N_TEST = len(images_test)
total_characters = N_TEST * N_DIGITS
lcr, ar, mismatches = model.accuracy(test_X, test_y, BATCH_SIZE, total_characters, N_TEST)
print("lcr: %f" % (lcr))
print("ar: %f" % (ar))

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
lcr: 0.787000
ar: 0.954000


In [20]:
logDir = model_savepath / "errors"
# avoid accidentaly overwriting old logfiles
counter = 1
while logDir.exists():
    print("folder %s already exists!"%(logDir))
    counter += 1
    logDir = model_savepath / ("errors%d"%(counter))
# save log
evalFunctions.log_errors(mismatches, labels_test, savePath=logDir)
# save accuracy
accFile = logDir / "accuracy.txt"
accFile.touch()
accFile.write_text("lcr: %f\nar: %f"%(lcr,ar))
# save what was generated
summaryFile = logDir / "about.txt"
summaryFile.touch()
summary = ""
summary += "targetCount = %d\n"%(targetCount)
summary += "Generated Digits:\n"
for digit, count in sorted(n_toGenerate.items()):
    summary += "%d:\t%d\n"%(digit, count)
summaryFile.write_text(summary)

folder D:\m2\evaluations\fcsrn\add_midstate2\errors already exists!
folder D:\m2\evaluations\fcsrn\add_midstate2\errors2 already exists!
created dir: D:\m2\evaluations\fcsrn\add_midstate2\errors3
wrote evalfile to: D:\m2\evaluations\fcsrn\add_midstate2\errors3\errors.txt
-------------------------------------
Total Characters
-------------------------------------
digit	count(total=5000)
0	1651
1	828
2	314
3	300
4	118
5	230
6	201
7	294
8	360
9	209
10	37
11	37
12	58
13	81
14	54
15	36
16	40
17	36
18	68
19	48
-------------------------------------
Total Errors
-------------------------------------
digits missing:	15
digits extra:	12
digits replaced:	209
-------------------------------------
Missing Digit Errors
-------------------------------------
	Digits by #missed:
digit	#missing(total=15)
0:	2
1:	3
3:	1
4:	1
5:	1
7:	1
8:	1
9:	1
10:	1
13:	1
14:	1
15:	1

	List of every missing-error:
missing	predicted	truth
0	[0 7 8 5] 	[0 0 7 8 5]
0	[0 7 8 5] 	[0 0 7 8 5]
1	[ 0  1  2 17] 	[0 1 2 1 1]
1	[ 

117